# Módulo 6: OCR e Reconhecimento de Texto## Objetivos de Aprendizagem- Compreender os fundamentos do OCR (Optical Character Recognition)- Conhecer ferramentas modernas como Tesseract e EasyOCR- Implementar soluções de reconhecimento de texto- Aplicar OCR em casos práticos do mercado- Entender limitações e desafios do OCR---## 6.1 Introdução ao OCR**OCR (Optical Character Recognition)** é uma tecnologia que converte imagens contendo texto em texto editável e pesquisável.![Introdução OCR](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo6/introducao_ocr.png)### Conceitos Fundamentais**Definição:**- **Conversão de imagem para texto**: Transformar pixels em caracteres- **Processamento de documentos**: Digitalização inteligente- **Extração de informação**: Dados estruturados de imagens- **Automação de processos**: Redução de trabalho manual**Componentes do OCR:**1. **Pré-processamento**: Melhoria da qualidade da imagem2. **Detecção de texto**: Localização de regiões com texto3. **Reconhecimento de caracteres**: Identificação de letras e números4. **Pós-processamento**: Correção e formatação do texto### Histórico e Evolução**Evolução Cronológica:**- **1950s-1960s**: Primeiros sistemas OCR comerciais- **1970s-1980s**: Melhoria na precisão e velocidade- **1990s-2000s**: Integração com computadores pessoais- **2010s**: Deep Learning revoluciona o campo- **2020s**: OCR baseado em redes neurais profundas![Evolução OCR](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo6/evolucao_ocr.png)**Marcos Importantes:**- **1951**: Primeiro sistema OCR comercial (IBM)- **1974**: Ray Kurzweil desenvolve primeiro OCR para cegos- **1995**: Tesseract OCR open source- **2018**: EasyOCR com deep learning- **2020**: PaddleOCR com alta precisão**Referências:**- [Optical Character Recognition - An Illustrated Guide to the Frontier - Casey & Lecolinet](https://ieeexplore.ieee.org/document/546110)

## 6.2 Demonstração Prática: OCR com Python

Vamos implementar e comparar diferentes ferramentas de OCR:


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import pytesseract
import easyocr
import time
import re

class OCRDemo:
    """Demonstração de diferentes ferramentas de OCR"""
    
    def __init__(self):
        self.reader = None
        
    def create_sample_text_images(self):
        """Cria imagens de exemplo com texto para demonstração"""
        
        images = {}
        
        # Imagem 1: Texto simples
        img1 = np.ones((200, 600, 3), dtype=np.uint8) * 255
        cv2.putText(img1, 'Hello World!', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
        cv2.putText(img1, 'OCR Demo', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2)
        images['simple_text'] = img1
        
        # Imagem 2: Texto com números
        img2 = np.ones((200, 600, 3), dtype=np.uint8) * 255
        cv2.putText(img2, 'Price: $29.99', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
        cv2.putText(img2, 'Date: 2024-01-15', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2)
        images['numbers_text'] = img2
        
        # Imagem 3: Texto com ruído
        img3 = np.ones((200, 600, 3), dtype=np.uint8) * 255
        cv2.putText(img3, 'Noisy Text', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
        cv2.putText(img3, 'Recognition', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2)
        
        # Adicionar ruído
        noise = np.random.randint(0, 50, img3.shape, dtype=np.uint8)
        img3 = cv2.add(img3, noise)
        images['noisy_text'] = img3
        
        # Imagem 4: Texto rotacionado
        img4 = np.ones((200, 600, 3), dtype=np.uint8) * 255
        cv2.putText(img4, 'Rotated Text', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
        cv2.putText(img4, 'Challenge', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2)
        
        # Rotacionar imagem
        center = (img4.shape[1] // 2, img4.shape[0] // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, 15, 1.0)
        img4 = cv2.warpAffine(img4, rotation_matrix, (img4.shape[1], img4.shape[0]))
        images['rotated_text'] = img4
        
        # Imagem 5: Texto pequeno
        img5 = np.ones((200, 600, 3), dtype=np.uint8) * 255
        cv2.putText(img5, 'Small Text', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        cv2.putText(img5, 'Hard to Read', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 1)
        images['small_text'] = img5
        
        return images
    
    def preprocess_image(self, img, method='basic'):
        """Pré-processa a imagem para melhorar OCR"""
        
        if method == 'basic':
            # Conversão para escala de cinza
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # Binarização
            _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
            return binary
        
        elif method == 'advanced':
            # Conversão para escala de cinza
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # Redução de ruído
            denoised = cv2.medianBlur(gray, 3)
            
            # Binarização adaptativa
            binary = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
            
            # Morfologia para limpeza
            kernel = np.ones((2, 2), np.uint8)
            cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
            
            return cleaned
        
        return img
    
    def ocr_with_tesseract(self, img, preprocess=True):
        """Executa OCR usando Tesseract"""
        
        try:
            if preprocess:
                processed_img = self.preprocess_image(img, 'basic')
            else:
                processed_img = img
            
            # Configurações do Tesseract
            config = '--oem 3 --psm 6 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789.,!?$: '
            
            # Executar OCR
            start_time = time.time()
            text = pytesseract.image_to_string(processed_img, config=config)
            processing_time = time.time() - start_time
            
            # Limpar texto
            text = text.strip()
            
            return {
                'text': text,
                'time': processing_time,
                'method': 'Tesseract'
            }
        
        except Exception as e:
            return {
                'text': f'Erro: {str(e)}',
                'time': 0,
                'method': 'Tesseract'
            }
    
    def ocr_with_easyocr(self, img, preprocess=False):
        """Executa OCR usando EasyOCR"""
        
        try:
            # Inicializar EasyOCR (apenas uma vez)
            if self.reader is None:
                self.reader = easyocr.Reader(['en'])
            
            if preprocess:
                processed_img = self.preprocess_image(img, 'advanced')
            else:
                processed_img = img
            
            # Executar OCR
            start_time = time.time()
            results = self.reader.readtext(processed_img)
            processing_time = time.time() - start_time
            
            # Extrair texto
            text_parts = []
            for (bbox, text, confidence) in results:
                if confidence > 0.5:  # Threshold de confiança
                    text_parts.append(text)
            
            text = ' '.join(text_parts)
            
            return {
                'text': text,
                'time': processing_time,
                'method': 'EasyOCR',
                'confidence': np.mean([conf for _, _, conf in results]) if results else 0
            }
        
        except Exception as e:
            return {
                'text': f'Erro: {str(e)}',
                'time': 0,
                'method': 'EasyOCR'
            }
    
    def simulate_ocr_results(self, img_name, img):
        """Simula resultados de OCR para demonstração"""
        
        # Simular resultados baseados no tipo de imagem
        if img_name == 'simple_text':
            tesseract_result = {
                'text': 'Hello World!\nOCR Demo',
                'time': 0.15,
                'method': 'Tesseract'
            }
            easyocr_result = {
                'text': 'Hello World! OCR Demo',
                'time': 0.25,
                'method': 'EasyOCR',
                'confidence': 0.95
            }
        
        elif img_name == 'numbers_text':
            tesseract_result = {
                'text': 'Price: $29.99\nDate: 2024-01-15',
                'time': 0.18,
                'method': 'Tesseract'
            }
            easyocr_result = {
                'text': 'Price: $29.99 Date: 2024-01-15',
                'time': 0.28,
                'method': 'EasyOCR',
                'confidence': 0.92
            }
        
        elif img_name == 'noisy_text':
            tesseract_result = {
                'text': 'Noisy Text\nRecognition',
                'time': 0.22,
                'method': 'Tesseract'
            }
            easyocr_result = {
                'text': 'Noisy Text Recognition',
                'time': 0.32,
                'method': 'EasyOCR',
                'confidence': 0.78
            }
        
        elif img_name == 'rotated_text':
            tesseract_result = {
                'text': 'Rotated Text\nChallenge',
                'time': 0.20,
                'method': 'Tesseract'
            }
            easyocr_result = {
                'text': 'Rotated Text Challenge',
                'time': 0.30,
                'method': 'EasyOCR',
                'confidence': 0.85
            }
        
        else:  # small_text
            tesseract_result = {
                'text': 'Small Text\nHard to Read',
                'time': 0.25,
                'method': 'Tesseract'
            }
            easyocr_result = {
                'text': 'Small Text Hard to Read',
                'time': 0.35,
                'method': 'EasyOCR',
                'confidence': 0.68
            }
        
        return tesseract_result, easyocr_result
    
    def visualize_ocr_results(self, images, results):
        """Visualiza resultados do OCR"""
        
        fig, axes = plt.subplots(3, 5, figsize=(20, 12))
        
        for i, (img_name, img) in enumerate(images.items()):
            # Imagem original
            axes[0, i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            axes[0, i].set_title(f'Original: {img_name}')
            axes[0, i].axis('off')
            
            # Imagem pré-processada
            processed = self.preprocess_image(img, 'basic')
            axes[1, i].imshow(processed, cmap='gray')
            axes[1, i].set_title('Pré-processada')
            axes[1, i].axis('off')
            
            # Resultados do OCR
            tesseract_result, easyocr_result = results[img_name]
            
            result_text = f"Tesseract:\n{tesseract_result['text'][:50]}...\n\nEasyOCR:\n{easyocr_result['text'][:50]}..."
            
            axes[2, i].text(0.05, 0.95, result_text, transform=axes[2, i].transAxes, 
                          fontsize=8, verticalalignment='top',
                          bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
            axes[2, i].set_title('Resultados OCR')
            axes[2, i].axis('off')
        
        plt.suptitle('Comparação de Ferramentas de OCR', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        return results
    
    def analyze_ocr_performance(self, results):
        """Analisa performance das ferramentas de OCR"""
        
        # Calcular métricas
        tesseract_times = []
        easyocr_times = []
        easyocr_confidences = []
        
        for img_name, (tesseract_result, easyocr_result) in results.items():
            tesseract_times.append(tesseract_result['time'])
            easyocr_times.append(easyocr_result['time'])
            if 'confidence' in easyocr_result:
                easyocr_confidences.append(easyocr_result['confidence'])
        
        # Visualizar métricas
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # Gráfico de tempo de processamento
        img_names = list(results.keys())
        x = np.arange(len(img_names))
        width = 0.35
        
        axes[0].bar(x - width/2, tesseract_times, width, label='Tesseract', color='lightblue')
        axes[0].bar(x + width/2, easyocr_times, width, label='EasyOCR', color='lightgreen')
        axes[0].set_title('Tempo de Processamento')
        axes[0].set_ylabel('Tempo (segundos)')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels([name.replace('_', ' ').title() for name in img_names], rotation=45)
        axes[0].legend()
        
        # Gráfico de confiança (EasyOCR)
        axes[1].bar(img_names, easyocr_confidences, color='lightcoral')
        axes[1].set_title('Confiança do EasyOCR')
        axes[1].set_ylabel('Confiança')
        axes[1].tick_params(axis='x', rotation=45)
        axes[1].set_ylim(0, 1)
        
        # Adicionar valores nas barras
        for i, v in enumerate(easyocr_confidences):
            axes[1].text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom')
        
        # Comparação de características
        characteristics = {
            'Velocidade': [np.mean(tesseract_times), np.mean(easyocr_times)],
            'Precisão': [0.85, 0.90],  # Valores simulados
            'Facilidade': [0.95, 0.80],  # Valores simulados
            'Suporte': [0.90, 0.85]  # Valores simulados
        }
        
        methods = ['Tesseract', 'EasyOCR']
        x = np.arange(len(methods))
        width = 0.2
        
        for i, (char, values) in enumerate(characteristics.items()):
            axes[2].bar(x + i*width, values, width, label=char)
        
        axes[2].set_title('Comparação de Características')
        axes[2].set_ylabel('Pontuação')
        axes[2].set_xticks(x + width * 1.5)
        axes[2].set_xticklabels(methods)
        axes[2].legend()
        axes[2].set_ylim(0, 1)
        
        plt.suptitle('Análise de Performance do OCR', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        # Resumo das métricas
        print(f"\n=== RESUMO DAS MÉTRICAS ===")
        print(f"Tesseract - Tempo Médio: {np.mean(tesseract_times):.2f}s")
        print(f"EasyOCR - Tempo Médio: {np.mean(easyocr_times):.2f}s")
        print(f"EasyOCR - Confiança Média: {np.mean(easyocr_confidences):.2f}")
        
        return {
            'tesseract_times': tesseract_times,
            'easyocr_times': easyocr_times,
            'easyocr_confidences': easyocr_confidences
        }

def demonstrate_ocr():
    """Demonstra diferentes ferramentas de OCR"""
    
    print("=== Demonstração de OCR ===")
    print("\nNota: Esta demonstração simula resultados de OCR para fins educacionais.")
    print("Em aplicações reais, use as ferramentas reais com imagens reais.")
    
    # Criar instância
    demo = OCRDemo()
    
    # Criar imagens de exemplo
    print("\nCriando imagens de exemplo...")
    images = demo.create_sample_text_images()
    
    # Simular resultados de OCR
    print("\nSimulando resultados de OCR...")
    results = {}
    for img_name, img in images.items():
        tesseract_result, easyocr_result = demo.simulate_ocr_results(img_name, img)
        results[img_name] = (tesseract_result, easyocr_result)
    
    # Visualizar resultados
    print("\nVisualizando resultados...")
    demo.visualize_ocr_results(images, results)
    
    # Analisar performance
    print("\nAnalisando performance...")
    metrics = demo.analyze_ocr_performance(results)
    
    return results, metrics

# Executar demonstração
ocr_results, ocr_metrics = demonstrate_ocr()

### Análise dos Resultados

**OCR Observado:**

1. **Tesseract**: Mais rápido, boa precisão para texto simples
2. **EasyOCR**: Mais lento, melhor precisão para texto complexo
3. **Pré-processamento**: Melhora significativamente os resultados
4. **Confiança**: EasyOCR fornece métricas de confiança

**Insights Importantes:**
- **Velocidade**: Tesseract é mais rápido
- **Precisão**: EasyOCR é mais preciso
- **Robustez**: EasyOCR lida melhor com ruído
- **Facilidade**: Tesseract é mais fácil de usar

**Referências:**
- [Tesseract OCR - Google](https://github.com/tesseract-ocr/tesseract)
- [EasyOCR - JaidedAI](https://github.com/JaidedAI/EasyOCR)


## 6.3 Ferramentas Modernas de OCR### Tesseract OCR**Características:**- **Open source**: Desenvolvido pelo Google- **Multi-idioma**: Suporte a 100+ idiomas- **Configurável**: Múltiplas opções de configuração- **Estável**: Ferramenta madura e confiável![Tesseract OCR](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo6/tesseract_ocr.png)**Vantagens:**- **Velocidade**: Processamento rápido- **Simplicidade**: Fácil de implementar- **Estabilidade**: Resultados consistentes- **Documentação**: Bem documentado**Desvantagens:**- **Precisão**: Limitada para texto complexo- **Ruído**: Sensível a ruído e distorções- **Configuração**: Requer ajuste fino- **Limitações**: Não funciona bem com texto rotacionado### EasyOCR**Características:**- **Deep Learning**: Baseado em redes neurais- **Multi-idioma**: Suporte a 80+ idiomas- **Confiança**: Fornece métricas de confiança- **Robustez**: Lida bem com texto complexo![EasyOCR](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo6/easyocr.png)**Vantagens:**- **Precisão**: Alta precisão para texto complexo- **Robustez**: Lida bem com ruído e distorções- **Confiança**: Métricas de confiança- **Flexibilidade**: Funciona com diferentes tipos de texto**Desvantagens:**- **Velocidade**: Mais lento que Tesseract- **Recursos**: Requer mais recursos computacionais- **Dependências**: Mais dependências- **Complexidade**: Mais complexo de configurar### PaddleOCR**Características:**- **Deep Learning**: Baseado em PaddlePaddle- **Multi-idioma**: Suporte a 80+ idiomas- **Precisão**: Alta precisão- **Velocidade**: Otimizado para velocidade![PaddleOCR](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo6/paddleocr.png)**Vantagens:**- **Precisão**: Alta precisão- **Velocidade**: Rápido- **Robustez**: Lida bem com texto complexo- **Suporte**: Bom suporte da comunidade**Desvantagens:**- **Complexidade**: Mais complexo- **Dependências**: Muitas dependências- **Documentação**: Documentação em chinês- **Suporte**: Suporte limitado em inglês**Referências:**- [Tesseract OCR - Google](https://github.com/tesseract-ocr/tesseract)- [EasyOCR - JaidedAI](https://github.com/JaidedAI/EasyOCR)- [PaddleOCR - PaddlePaddle](https://github.com/PaddlePaddle/PaddleOCR)

## 6.4 Pré-processamento para OCR### Técnicas de Pré-processamento**1. Conversão para Escala de Cinza:**- **Objetivo**: Reduzir dimensionalidade- **Método**: Média ponderada dos canais RGB- **Fórmula**: Gray = 0.299×R + 0.587×G + 0.114×B- **Benefício**: Reduz ruído e melhora processamento**2. Binarização:**- **Objetivo**: Separar texto do fundo- **Métodos**: Threshold global, adaptativo, Otsu- **Threshold Global**: Valor fixo para todos os pixels- **Threshold Adaptativo**: Valor baseado na vizinhança- **Otsu**: Valor ótimo baseado na distribuição![Técnicas de Pré-processamento](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo6/tecnicas_preprocessamento.png)**3. Redução de Ruído:**- **Filtro Gaussiano**: Suavização com kernel gaussiano- **Filtro Mediano**: Remove ruído impulsivo- **Morfologia**: Operações de abertura e fechamento- **Benefício**: Melhora qualidade da imagem**4. Correção de Inclinação:**- **Detecção**: Análise de projeções horizontais- **Correção**: Rotação baseada no ângulo detectado- **Métodos**: Hough transform, análise de linhas- **Benefício**: Melhora precisão do OCR### Demonstração Prática: Pré-processamentoVamos implementar e visualizar diferentes técnicas de pré-processamento:

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

class PreprocessingDemo:
    """Demonstração de técnicas de pré-processamento para OCR"""
    
    def __init__(self):
        pass
    
    def create_challenging_image(self):
        """Cria uma imagem desafiadora para OCR"""
        
        # Criar imagem base
        img = np.ones((300, 600, 3), dtype=np.uint8) * 200
        
        # Adicionar texto
        cv2.putText(img, 'Challenging OCR', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
        cv2.putText(img, 'Preprocessing Demo', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2)
        cv2.putText(img, 'Noise + Rotation', (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 2)
        
        # Adicionar ruído
        noise = np.random.randint(0, 50, img.shape, dtype=np.uint8)
        img = cv2.add(img, noise)
        
        # Adicionar inclinação
        center = (img.shape[1] // 2, img.shape[0] // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, 5, 1.0)
        img = cv2.warpAffine(img, rotation_matrix, (img.shape[1], img.shape[0]))
        
        return img
    
    def apply_preprocessing_techniques(self, img):
        """Aplica diferentes técnicas de pré-processamento"""
        
        results = {}
        
        # 1. Imagem original
        results['original'] = img
        
        # 2. Conversão para escala de cinza
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        results['grayscale'] = gray
        
        # 3. Binarização global
        _, binary_global = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        results['binary_global'] = binary_global
        
        # 4. Binarização Otsu
        _, binary_otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        results['binary_otsu'] = binary_otsu
        
        # 5. Binarização adaptativa
        binary_adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        results['binary_adaptive'] = binary_adaptive
        
        # 6. Redução de ruído
        denoised = cv2.medianBlur(gray, 3)
        results['denoised'] = denoised
        
        # 7. Morfologia
        kernel = np.ones((2, 2), np.uint8)
        morphed = cv2.morphologyEx(binary_otsu, cv2.MORPH_CLOSE, kernel)
        results['morphed'] = morphed
        
        # 8. Correção de inclinação
        # Detectar linhas
        edges = cv2.Canny(gray, 50, 150)
        lines = cv2.HoughLines(edges, 1, np.pi/180, threshold=100)
        
        if lines is not None:
            # Calcular ângulo médio
            angles = []
            for line in lines:
                rho, theta = line[0]
                angle = theta * 180 / np.pi
                if 45 < angle < 135:  # Linhas horizontais
                    angles.append(angle - 90)
            
            if angles:
                avg_angle = np.mean(angles)
                # Corrigir inclinação
                center = (img.shape[1] // 2, img.shape[0] // 2)
                rotation_matrix = cv2.getRotationMatrix2D(center, -avg_angle, 1.0)
                corrected = cv2.warpAffine(gray, rotation_matrix, (img.shape[1], img.shape[0]))
                results['corrected'] = corrected
            else:
                results['corrected'] = gray
        else:
            results['corrected'] = gray
        
        return results
    
    def visualize_preprocessing_results(self, results):
        """Visualiza resultados do pré-processamento"""
        
        fig, axes = plt.subplots(2, 4, figsize=(20, 10))
        
        # Títulos e descrições
        titles = {
            'original': 'Imagem Original',
            'grayscale': 'Escala de Cinza',
            'binary_global': 'Binarização Global',
            'binary_otsu': 'Binarização Otsu',
            'binary_adaptive': 'Binarização Adaptativa',
            'denoised': 'Redução de Ruído',
            'morphed': 'Morfologia',
            'corrected': 'Correção de Inclinação'
        }
        
        # Plotar resultados
        for i, (key, img) in enumerate(results.items()):
            row = i // 4
            col = i % 4
            
            if len(img.shape) == 3:
                axes[row, col].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            else:
                axes[row, col].imshow(img, cmap='gray')
            
            axes[row, col].set_title(titles[key])
            axes[row, col].axis('off')
        
        plt.suptitle('Técnicas de Pré-processamento para OCR', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        return results
    
    def analyze_preprocessing_quality(self, results):
        """Analisa qualidade do pré-processamento"""
        
        # Calcular métricas de qualidade
        quality_metrics = {}
        
        for key, img in results.items():
            if len(img.shape) == 3:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            else:
                gray = img
            
            # Calcular contraste
            contrast = np.std(gray)
            
            # Calcular nitidez (Laplaciano)
            laplacian = cv2.Laplacian(gray, cv2.CV_64F)
            sharpness = np.var(laplacian)
            
            # Calcular uniformidade
            uniformity = 1 - np.std(gray) / np.mean(gray)
            
            quality_metrics[key] = {
                'contrast': contrast,
                'sharpness': sharpness,
                'uniformity': uniformity
            }
        
        # Visualizar métricas
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # Gráfico de contraste
        keys = list(quality_metrics.keys())
        contrasts = [quality_metrics[key]['contrast'] for key in keys]
        
        axes[0].bar(keys, contrasts, color='lightblue')
        axes[0].set_title('Contraste')
        axes[0].set_ylabel('Desvio Padrão')
        axes[0].tick_params(axis='x', rotation=45)
        
        # Gráfico de nitidez
        sharpnesses = [quality_metrics[key]['sharpness'] for key in keys]
        
        axes[1].bar(keys, sharpnesses, color='lightgreen')
        axes[1].set_title('Nitidez')
        axes[1].set_ylabel('Variância do Laplaciano')
        axes[1].tick_params(axis='x', rotation=45)
        
        # Gráfico de uniformidade
        uniformities = [quality_metrics[key]['uniformity'] for key in keys]
        
        axes[2].bar(keys, uniformities, color='lightcoral')
        axes[2].set_title('Uniformidade')
        axes[2].set_ylabel('Uniformidade')
        axes[2].tick_params(axis='x', rotation=45)
        
        plt.suptitle('Métricas de Qualidade do Pré-processamento', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        # Resumo das métricas
        print(f"\n=== RESUMO DAS MÉTRICAS ===")
        for key, metrics in quality_metrics.items():
            print(f"{key}: Contraste={metrics['contrast']:.1f}, Nitidez={metrics['sharpness']:.1f}, Uniformidade={metrics['uniformity']:.2f}")
        
        return quality_metrics

def demonstrate_preprocessing():
    """Demonstra técnicas de pré-processamento"""
    
    print("=== Demonstração de Pré-processamento para OCR ===")
    
    # Criar instância
    demo = PreprocessingDemo()
    
    # Criar imagem desafiadora
    print("\nCriando imagem desafiadora...")
    img = demo.create_challenging_image()
    
    # Aplicar técnicas de pré-processamento
    print("\nAplicando técnicas de pré-processamento...")
    results = demo.apply_preprocessing_techniques(img)
    
    # Visualizar resultados
    print("\nVisualizando resultados...")
    demo.visualize_preprocessing_results(results)
    
    # Analisar qualidade
    print("\nAnalisando qualidade...")
    metrics = demo.analyze_preprocessing_quality(results)
    
    return results, metrics

# Executar demonstração
preprocessing_results, preprocessing_metrics = demonstrate_preprocessing()

### Análise dos Resultados

**Pré-processamento Observado:**

1. **Escala de Cinza**: Reduz ruído e melhora processamento
2. **Binarização Otsu**: Melhor separação texto/fundo
3. **Binarização Adaptativa**: Lida bem com iluminação variável
4. **Redução de Ruído**: Melhora qualidade da imagem
5. **Morfologia**: Limpa artefatos
6. **Correção de Inclinação**: Melhora precisão do OCR

**Insights Importantes:**
- **Sequência**: Ordem das operações é importante
- **Parâmetros**: Ajuste fino necessário
- **Qualidade**: Métricas objetivas de qualidade
- **Aplicação**: Depende do tipo de imagem

**Referências:**
- [Optical Character Recognition - An Illustrated Guide to the Frontier - Casey & Lecolinet](https://ieeexplore.ieee.org/document/546110)


## 6.5 Aplicações Práticas do OCR### Casos de Uso Reais**1. Digitalização de Documentos:**- **Contratos**: Extração de informações de contratos- **Faturas**: Processamento automático de faturas- **Relatórios**: Digitalização de relatórios impressos- **Exemplo**: Adobe Acrobat, Google Drive**2. Automação de Processos:**- **Entrada de dados**: Automatização de entrada manual- **Validação**: Verificação automática de documentos- **Classificação**: Categorização automática- **Exemplo**: RPA (Robotic Process Automation)**3. Acessibilidade:**- **Leitores de tela**: Conversão de texto para áudio- **Tradução**: Tradução automática de texto- **Navegação**: Ajuda para pessoas com deficiência visual- **Exemplo**: Google Lens, Microsoft Seeing AI**4. Mobile e IoT:**- **Apps móveis**: Reconhecimento de texto em tempo real- **Dispositivos IoT**: Processamento de texto em dispositivos- **Realidade aumentada**: Sobreposição de informações- **Exemplo**: Google Translate, Apple Live Text![Aplicações Práticas OCR](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo6/aplicacoes_praticas_ocr.png)### Desafios e Limitações**1. Qualidade da Imagem:**- **Resolução**: Imagens de baixa resolução- **Iluminação**: Iluminação inadequada- **Ruído**: Ruído e artefatos- **Distorção**: Distorções ópticas**2. Tipos de Texto:**- **Fontes**: Fontes não padrão- **Tamanho**: Texto muito pequeno- **Estilo**: Texto manuscrito- **Idioma**: Idiomas não suportados**3. Contexto:**- **Layout**: Layout complexo- **Orientação**: Texto rotacionado- **Sobreposição**: Texto sobreposto- **Fundo**: Fundo complexo![Desafios OCR](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo6/desafios_ocr.png)**Referências:**- [Optical Character Recognition - An Illustrated Guide to the Frontier - Casey & Lecolinet](https://ieeexplore.ieee.org/document/546110)

## Resumo do Módulo 6

### Principais Conceitos Abordados

1. **Introdução ao OCR**
   - Conceitos fundamentais
   - Componentes do sistema
   - Histórico e evolução

2. **Ferramentas Modernas**
   - Tesseract OCR
   - EasyOCR
   - PaddleOCR
   - Comparação de características

3. **Pré-processamento**
   - Técnicas de melhoria
   - Binarização e redução de ruído
   - Correção de inclinação
   - Métricas de qualidade

4. **Aplicações Práticas**
   - Casos de uso reais
   - Desafios e limitações
   - Tendências futuras

### Demonstrações Práticas

**1. Comparação de Ferramentas:**
   - Tesseract vs EasyOCR
   - Análise de performance
   - Métricas de qualidade

**2. Pré-processamento:**
   - Técnicas de melhoria
   - Visualização de resultados
   - Análise de qualidade

### Próximos Passos

No próximo módulo, exploraremos **GANs e VAEs para Geração Sintética de Imagens**, onde aprenderemos sobre modelos generativos.

### Referências Principais

- [Optical Character Recognition - An Illustrated Guide to the Frontier - Casey & Lecolinet](https://ieeexplore.ieee.org/document/546110)
- [Tesseract OCR - Google](https://github.com/tesseract-ocr/tesseract)
- [EasyOCR - JaidedAI](https://github.com/JaidedAI/EasyOCR)

---

**Próximo Módulo**: GANs e VAEs para Geração Sintética de Imagens
